# Training Draft

#### Xiao Xin

In [53]:
class Segment:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.vector = [e - s for s, e in zip(start, end)]
        self.length = sum((s - e) * (s - e) for s, e in zip(start, end))
    
    def __iter__(self):
        i , j = self.start
        dx, dy = self.vector
        if dx == 0:
            dx, dy = 0, 1
        else:
            dx, dy = 1, dy / float(dx)
        while i <= self.end[0] or j <= self.end[1]:
            yield i, int(j)
            i, j = i + dx, j + dy
    
    def distance(self, point):
        point_vector = [e - s for s, e in zip(self.start, point)]
        dot_product = sum(a * b for a, b in zip(point_vector, self.vector))
        if dot_product < 0: 
            return sum((s - e) * (s - e) for s, e in zip(self.start, point))
        if dot_product >= self.length:
            return sum((s - e) * (s - e) for s, e in zip(self.end, point))
        projection = dot_product * dot_product / float(self.length)
        distance = sum(x * x for x in point_vector) - projection
        return distance

class Reducer:
    def __init__(self, skeleton):
        self.skeleton = Mapper(skeleton)
    
    def reduce(self, graph):
        # graph -> Mapper
        return sum(self.__reduce_block(graph, block_id) for block_id in xrange(SIDE * SIDE))
    
    def __reduce_block(self, graph, block_id):
        # return block MSE
        skeleton_block = self.skeleton.get(block_id)
        graph_block = graph.get(block_id)
        
        result = 0
        for segment in graph_block:
            for point in segment:
                result += min(norm.distance(point) for norm in skeleton_block)
        return result